# Pull from AWS S3

In [1]:
import boto3
from io import BytesIO
import logging
import pandas as pd

# gitignore file security AWS password access
# rootkey.csv create in webconsole AWS -> MYNAME -> Info. Id. Secu. -> IAM -> Access Key 
rootkey = pd.read_csv('src/rootkey.csv',header=None, sep="=").T
rootkey.columns = rootkey.iloc[0]
rootkey = rootkey.drop([0])

# Sensitive informations Account
AWSAccessKeyId = rootkey['AWSAccessKeyId'].iloc[0]
AWSSecretKey = rootkey['AWSSecretKey'].iloc[0]

# Open session 
session = boto3.Session(aws_access_key_id=AWSAccessKeyId, 
                        aws_secret_access_key=AWSSecretKey)

# Run session
s3 = session.resource("s3")

# Upload file 
bucket_name = 'compartiment-bucket-s3-jedha-rg'
name = 'booking_weather_dataset.csv' 
obj = s3.Object(bucket_name, name)

# Reading the File as String With Encoding
file_content = BytesIO(obj.get()['Body'].read())

dataframe = pd.read_csv(file_content)

In [2]:
dataframe.columns

Index(['hotel_name', 'hotel_adress', 'hotel_note', 'hotel_describe',
       'hotel_latitude', 'hotel_longitude', 'url_hostel', 'city', 'latitude',
       'longitude', 'adress', 'weather_code', 'felt_temperature',
       'cloud_coverage', 'precipitation', 'wind', 'description'],
      dtype='object')

## Do a selection of nice out cities

In [3]:
# Drop heat wave °C
dataframe = dataframe[dataframe['felt_temperature'] < 36 ]

# Sort by best weather 
dataframe = dataframe.sort_values(by=['cloud_coverage', 'precipitation', 'wind'], ascending=True)

# Select the 5 first cities weather
best_weather_cities = dataframe['city'].unique()[0:5]
print(best_weather_cities)

# Drop other cities
dataframe = dataframe[dataframe['city'].isin(best_weather_cities)]

['Colmar' 'Nimes' 'Strasbourg' 'Toulouse' 'Dijon']


## Top 20 of hostels

In [4]:
# Best hotels 
dataframe = dataframe.sort_values(by='hotel_note', ascending=False)

# Keep 20 first hotels by city
best_hotels = pd.DataFrame()

for city in best_weather_cities:
    city_dataframe = dataframe[dataframe['city'] == city]

    if len(city_dataframe) > 20:
        city_dataframe = city_dataframe[0:20]
        
    best_hotels = pd.concat([best_hotels, city_dataframe])

dataframe = best_hotels

## Plot the top 5 cities, 20 hostels per city

In [5]:
import plotly.express as px
import plotly.io as pio
import re

for city in best_weather_cities:
    dataframe_city = dataframe[dataframe['city'] == city]

    description = str(dataframe_city['description'].iloc[0])
    felt_temperature = str(round(dataframe_city['felt_temperature'].iloc[0]))

    fig = px.scatter_mapbox(dataframe_city, 
                            lat="hotel_latitude", lon="hotel_longitude", 
                            color="hotel_note", zoom =12, 
                            center= {'lat': dataframe_city['latitude'].mean(), 
                                     'lon': dataframe_city['longitude'].mean()},
                            mapbox_style="carto-positron",
                            title= f"{city} : {description}, {felt_temperature}°C",
                            hover_name='hotel_name',
                            text= 'hotel_adress',
                            hover_data=['hotel_note'],
                            labels={'hotel_name':'Name','hotel_note':'Rating', 'hotel_adress': 'Adress', 
                                    'hotel_latitude':'Lat', 'hotel_longitude':'Lon'},
                            color_continuous_scale = 'sunset',
                            size="hotel_note", size_max= 15,
                            width=800, height=600,
                            range_color = [0,10])

    cloud_coverage = str(round(dataframe_city['cloud_coverage'].iloc[0]))
    precipitation = str(round(dataframe_city['precipitation'].iloc[0]))
    wind = str(round(dataframe_city['wind'].iloc[0]*3.6))
    text = f"<br>The cloud coverage is {cloud_coverage}%, it's raining {precipitation} mm/h and the wind blowing at {wind} km/h"

    fig.update_layout(annotations=[dict(text=text, showarrow=False,
                                     x=0, y=-0.1, yref="paper", align="left")])
     
    fig.show()

    dataframe_hotels = dataframe_city.drop(columns=['hotel_latitude','hotel_adress', 'hotel_note', 'hotel_longitude', 'city', 'latitude',
    'longitude', 'adress', 'weather_code', 'felt_temperature', 'cloud_coverage', 'precipitation', 'wind', 'description'])
    dataframe_hotels = dataframe_hotels.reset_index().drop(columns=['index'])
    dataframe_hotels['url_hostel'] = dataframe_hotels['url_hostel'].apply(lambda x : re.sub(r'\?aid=\S*', '', x))
    dataframe_hotels['hotel_describe'] = dataframe_hotels['hotel_describe'].apply(lambda x : re.sub(r'<a href=(.*?)<\/a>', '', x))
    dataframe_hotels['hotel_describe'] = dataframe_hotels['hotel_describe'].apply(lambda x : re.sub(r'Vous pouvez(.*?)économiser.', '', x))
    dataframe_hotels = dataframe_hotels.reindex(columns=['hotel_name','url_hostel','hotel_describe'])

    display(dataframe_hotels)
    


,hotel_name,url_hostel,hotel_describe
0,les appartements saint martin,https://www.booking.com/hotel/fr/les-apparteme...,Bénéficiant d’un emplacement pratique dans l...
1,SERENITY Colmar - Gîtes de Prestige 4 et 5 éto...,https://www.booking.com/hotel/fr/serenity-colm...,"Situé à Colmar, à proximité de la collégiale..."
2,At home in the center of Colmar,https://www.booking.com/hotel/fr/at-home-on-co...,"Doté d'une connexion Wi-Fi gratuite, l'héber..."
3,Le petit nid de Max et Sophie,https://www.booking.com/hotel/fr/le-petit-nid-...,"Situé à Colmar, à moins de 400 mètres de la ..."
4,Colmar Hotel,https://www.booking.com/hotel/fr/colmar.fr.html,"Le Colmar Hotel vous accueille à Colmar, à 1..."
5,PAUL & PIA - Welcome Home Hotel,https://www.booking.com/hotel/fr/kyriad-colmar...,L’Hôtel Centre Gare est situé à 500 mètres d...
6,comfort expo colmar,https://www.booking.com/hotel/fr/comfort-expo-...,Installé à 4 km du centre-ville de Colmar et à...
7,ColmarAppart Vauban,https://www.booking.com/hotel/fr/colmarappart-...,Le ColmarAppart Vauban vous accueille à Colm...
8,Suites Residences Spa,https://www.booking.com/hotel/fr/suites-reside...,"Situé à 200 mètres de la Maison des têtes, l..."
9,cotoon colmar8,https://www.booking.com/hotel/fr/cotoon-colmar...,Le COLMARAPPART Grand'Rue bénéficie d'un emp...


,hotel_name,url_hostel,hotel_describe
0,le jungle spa,https://www.booking.com/hotel/fr/le-jungle-spa...,"Featuring a spa bath, Le Jungle SPA **** is lo..."
1,tres belle rotonde deco boheme tennis parking,https://www.booking.com/hotel/fr/tres-belle-ro...,"Offrant une vue sur le jardin, le Très Belle r..."
2,le chant des oiseaux nimes3,https://www.booking.com/hotel/fr/le-chant-des-...,"Situé à Nîmes, à moins de 5 km du parc des exp..."
3,antichambre,https://www.booking.com/hotel/fr/antichambre.f...,"L'Antichambre se situe à Nîmes, à 1,9 km du pa..."
4,Guest House Encantada,https://www.booking.com/hotel/fr/encantadahome...,"Offrant une vue sur sa piscine extérieure, l..."
5,mas d 39 escattes,https://www.booking.com/hotel/fr/mas-d-39-esca...,Situé à 8 km du parc des expositions Parc Expo...
6,margaret chouleur,https://www.booking.com/hotel/fr/margaret-chou...,"Situé à Nîmes, le Margaret - Hôtel Chouleur pr..."
7,joli f2 proche gare arenes avec terrasse privee,https://www.booking.com/hotel/fr/joli-f2-proch...,"Situé à Nîmes, à 1,4 km du parc des expositi..."
8,ostium porte de,https://www.booking.com/hotel/fr/ostium-porte-...,Doté d'une connexion Wi-Fi gratuite et offrant...
9,Jardins Secrets,https://www.booking.com/hotel/fr/jardins-secre...,"Situé à Nîmes, l'hôtel Jardins Secrets occupe ..."


,hotel_name,url_hostel,hotel_describe
0,golden square strasbourg,https://www.booking.com/hotel/fr/golden-square...,"Situé dans le centre de Strasbourg, à 300 mè..."
1,L'Appart du Quai avec Terrasse,https://www.booking.com/hotel/fr/l-39-appart-d...,L'Appart du Quai avec Terrasse est un héberg...
2,carpe diem home,https://www.booking.com/hotel/fr/carpe-diem-ho...,"Proposant des hébergements avec une cuisine,..."
3,Les appartements Place de la Cathédrale,https://www.booking.com/hotel/fr/apartement-a-...,L'Apartement Place De La Cathedrale se situe...
4,Naeva Feng Shui,https://www.booking.com/hotel/fr/magnifique-ap...,Le Naeva Feng Shui vous accueille à Strasbourg...
5,Hôtel LÉONOR the place to live,https://www.booking.com/hotel/fr/leonor-the-pl...,"Situé à Strasbourg, à 200 mètres du marché d..."
6,embassy corner,https://www.booking.com/hotel/fr/embassy-corne...,Doté d'un patio avec vue sur le jardin et d'...
7,Hôtel Tandem - Boutique Hôtel,https://www.booking.com/hotel/fr/ha-tel-nid-de...,Ce boutique hôtel respectueux de l’environne...
8,HANNONG Hotel & Wine Bar,https://www.booking.com/hotel/fr/tihannong.fr....,L'Hôtel Hannong est situé au cœur de Strasbo...
9,k strasbourg,https://www.booking.com/hotel/fr/k-strasbourg....,"Situé à Strasbourg, le K Hotel dispose d’une s..."


,hotel_name,url_hostel,hotel_describe
0,le plaisancier t3 entierement renove de 60m3,https://www.booking.com/hotel/fr/le-plaisancie...,"Situé à Toulouse, à 1,8 km du parc des exposit..."
1,t2 palais de justice de 48 m2 en chartreuse to...,https://www.booking.com/hotel/fr/t2-palais-de-...,Le T2 - Palais de justice de 48 m2 en chartreu...
2,NEW Le Mosaïque - Saint Cyprien - Parking Privé,https://www.booking.com/hotel/fr/new-le-mosaiq...,"Offrant une vue sur la ville, le NEW Le Mosaïq..."
3,Charmant Appartement en Hypercentre,https://www.booking.com/hotel/fr/charmant-appa...,Le Charmant Appartement en Hypercentre propo...
4,Les Loges des Chalets,https://www.booking.com/hotel/fr/les-loges-des...,"Situé à 5 km du stade de Toulouse, l'établisse..."
5,Charming Apartment 1 bedroom - Purpan Cartouch...,https://www.booking.com/hotel/fr/charming-apar...,"Situé à Toulouse, à moins de 1 km du Zénith, à..."
6,Le Saint-Aubin by Cocoonr,https://www.booking.com/hotel/fr/le-saint-aubi...,Le Saint-Aubin by Cocoonr propose un hébergeme...
7,"T3 Spacieux, Renové en 2020 -Pkg-métro",https://www.booking.com/hotel/fr/spacieux-calm...,"Le T3 Spacieux, Renové en 2020 -Pkg-métro es..."
8,Ibis Styles Toulouse Capitole,https://www.booking.com/hotel/fr/du-taur.fr.html,L'Ibis Styles Toulouse Capitole est situé dans...
9,heliot,https://www.booking.com/hotel/fr/heliot.fr.html,L'Hôtel Héliot vous accueille dans un établiss...


,hotel_name,url_hostel,hotel_describe
0,La suite Caudalie - Charme & Cachet,https://www.booking.com/hotel/fr/la-suite-caud...,"Situé à Dijon, à 400 mètres de l'église Saint-..."
1,the duchess suite heart of dijon,https://www.booking.com/hotel/fr/the-duchess-s...,"Situé en plein centre de Dijon, à proximité de..."
2,Le Lion des Ducs : centre historique de Dijon,https://www.booking.com/hotel/fr/le-lion-des-d...,Doté d'une connexion Wi-Fi gratuite et offrant...
3,the duke suite heart of dijon with terrace,https://www.booking.com/hotel/fr/the-duke-suit...,"Situé à Dijon, à moins de 500 mètres de l'égli..."
4,bel appartement place saint joseph 4 a 5 pers ...,https://www.booking.com/hotel/fr/bel-apparteme...,Le Bel Appartement Rénové Garage 7mn Centre Vi...
5,L'historique au cœur de ville,https://www.booking.com/hotel/fr/l-39-historiq...,Doté d'une connexion Wi-Fi et d'un parking pri...
6,La Chambre Ducale Plein Coeur historique,https://www.booking.com/hotel/fr/plein-coeur-h...,L'établissement La Chambre Ducale Plein Coeur ...
7,appartement le louvre,https://www.booking.com/hotel/fr/appartement-l...,"Offrant une vue sur le jardin, l'Appartement L..."
8,56-chabot-charny,https://www.booking.com/hotel/fr/56-chabot-cha...,Le 56-chabot-charny est situé dans le quartier...
9,"Dijon : plein coeur de ville, à 2 pas du Tram",https://www.booking.com/hotel/fr/dijon-plein-c...,"Le Dijon: plein coeur de ville, à 2 pas du Tra..."
